In [14]:
import numpy as np
import pandas as pd

In [15]:
data = pd.read_csv("./dataset/clusterisation_dataset.csv")
data.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,credit_card,...,268.0,4.0,500.0,19.0,8.0,13.0,housewares,3149,sao paulo,SP
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3.0,voucher,...,268.0,4.0,500.0,19.0,8.0,13.0,housewares,3149,sao paulo,SP
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2.0,voucher,...,268.0,4.0,500.0,19.0,8.0,13.0,housewares,3149,sao paulo,SP
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,boleto,...,178.0,1.0,400.0,19.0,13.0,19.0,perfumery,47813,barreiras,BA
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,credit_card,...,232.0,1.0,420.0,24.0,19.0,21.0,auto,75265,vianopolis,GO


---
### Начинаем работать с фитами

---
### payment_installments

В наборах данных электронной коммерции (например, Olist — бразильская платформа) поле payment_installments часто встречается в таблицах, связанных с платежами. Оно помогает понять, как клиенты предпочитают платить за товары:

Одним платежом (payment_installments = 1) или в рассрочку (payment_installments > 1).

Какие категории товаров чаще покупают в рассрочку.

In [16]:
# payment_installments - рассрочка одним платежом (payment_installments = 1) или в рассрочку (payment_installments > 1)
data["payment_installments"].value_counts()

payment_installments
1.0     57663
2.0     13483
3.0     11570
4.0      7852
10.0     6761
5.0      5938
8.0      4963
6.0      4553
7.0      1781
9.0       711
12.0      163
15.0       90
18.0       38
24.0       34
11.0       25
20.0       20
13.0       18
14.0       15
17.0        7
16.0        7
21.0        6
0.0         3
23.0        1
22.0        1
Name: count, dtype: int64

In [17]:
# приведем к бинарному типу
data["payment_installments"] = data["payment_installments"].apply(lambda x: 1 if x > 1 else 0)
data["payment_installments"].value_counts()

payment_installments
1    58037
0    57669
Name: count, dtype: int64

---
### payment_sequential
Определение :

Поле payment_sequential представляет собой порядковый номер (или идентификатор) платежа в последовательности платежей, связанных с одним заказом.

Например:

Если заказ оплачивается несколькими способами (например, частично банковской картой, частично наличными), каждый способ оплаты может быть представлен отдельным платежом.

В этом случае payment_sequential будет нумеровать эти платежи: 1, 2, 3 и т.д.

In [18]:
# Дропнем фит так как для разметки клиента нам скорее нужна агрегированная сумма покупки, 
# следовательно платежная последовательность мимо, туда же и payment_type так как часто привязан к payment_sequential
data = data.drop(columns=["payment_sequential", "payment_type"])

----
### payment_value
Агрегируем по клиенту, возвращаем в основной фрейм дропаем старый

In [19]:
data.head() 


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_installments,payment_value,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,0,18.12,...,268.0,4.0,500.0,19.0,8.0,13.0,housewares,3149,sao paulo,SP
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,0,2.00,...,268.0,4.0,500.0,19.0,8.0,13.0,housewares,3149,sao paulo,SP
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,0,18.59,...,268.0,4.0,500.0,19.0,8.0,13.0,housewares,3149,sao paulo,SP
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,0,141.46,...,178.0,1.0,400.0,19.0,13.0,19.0,perfumery,47813,barreiras,BA
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,179.12,...,232.0,1.0,420.0,24.0,19.0,21.0,auto,75265,vianopolis,GO


In [26]:
customer_transactions = data.groupby(by=["customer_id", "order_id"])[
    "payment_value"].sum()
customer_transactions = customer_transactions.reset_index()
customer_transactions

,customer_id,order_id,payment_value
0,00012a2ce6f8dcda20d059ce98491703,5f79b5b0931d63f1a42989eb65b9da6e,114.74
1,000161a058600d5901f007fab4c27140,a44895d095d7e0702b6a162fa2dbeced,67.41
2,0001fd6190edaaf884bcaf3d49edf079,316a104623542e4d75189bb372bc5f8d,195.42
3,0002414f95344307404f0ace7a26f1d5,5825ce2e88d5346438686b0bba99e5ee,179.35
4,000379cdec625522490c315e70c7a9fb,0ab7fb08086d4af9141453c91878ed7a,107.01
...,...,...,...
96456,fffcb937e9dd47a13f05ecb8290f4d3e,620ddc9fbe3e445676a8e71d4a830821,91.91
96457,fffecc9f79fd8c764f843e9951b11341,814d6a3a7c0b32b2ad929ac6328124e9,81.36
96458,fffeda5b6d849fbd39689bb92087f431,8c855550908247a7eff50281b92167a8,63.13
96459,ffff42319e9b2d713724ae527742af25,83b5fc912b2862c5046555ded1483ae9,214.13


In [32]:
customer_transactions[(customer_transactions["customer_id"] == "9ef432eb6251297304e76186b10a928d") & (
    customer_transactions["order_id"] == "e481f51cbdc54678b7cc49136f2d6af7")]

,customer_id,order_id,payment_value
59942,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,38.71


In [35]:
data_stage_1 = pd.merge(left=customer_transactions, right=data, left_on=[
                        "customer_id", "order_id"], right_on=["customer_id", "order_id"], how="left")

data_stage_2 = data_stage_1.drop(columns=["payment_value_y"])

data_stage_2.rename(columns={"payment_value_y": "payment_sum"})

data_stage_2.head()

,customer_id,order_id,payment_value_x,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_zip_code_prefix,customer_city,customer_state
0,00012a2ce6f8dcda20d059ce98491703,5f79b5b0931d63f1a42989eb65b9da6e,114.74,delivered,2017-11-14 16:08:26,2017-11-14 16:35:32,2017-11-17 15:32:08,2017-11-28 15:41:30,2017-12-04 00:00:00,1,...,889.0,1.0,4267.0,46.0,39.0,22.0,toys,6273,osasco,SP
1,000161a058600d5901f007fab4c27140,a44895d095d7e0702b6a162fa2dbeced,67.41,delivered,2017-07-16 09:40:32,2017-07-16 09:55:12,2017-07-19 19:09:37,2017-07-25 18:57:33,2017-08-04 00:00:00,1,...,2563.0,2.0,150.0,16.0,2.0,11.0,health_beauty,35550,itapecerica,MG
2,0001fd6190edaaf884bcaf3d49edf079,316a104623542e4d75189bb372bc5f8d,195.42,delivered,2017-02-28 11:06:43,2017-02-28 11:15:20,2017-03-01 15:24:20,2017-03-06 08:57:49,2017-03-22 00:00:00,1,...,1042.0,2.0,750.0,21.0,23.0,35.0,baby,29830,nova venecia,ES
3,0002414f95344307404f0ace7a26f1d5,5825ce2e88d5346438686b0bba99e5ee,179.35,delivered,2017-08-16 13:09:20,2017-08-17 03:10:27,2017-08-19 11:34:29,2017-09-13 20:06:02,2017-09-14 00:00:00,0,...,840.0,1.0,1600.0,40.0,24.0,30.0,cool_stuff,39664,mendonca,MG
4,000379cdec625522490c315e70c7a9fb,0ab7fb08086d4af9141453c91878ed7a,107.01,delivered,2018-04-02 13:42:17,2018-04-04 03:10:19,2018-04-04 18:11:09,2018-04-13 20:21:08,2018-04-18 00:00:00,0,...,364.0,2.0,800.0,45.0,10.0,34.0,bed_bath_table,4841,sao paulo,SP


In [36]:
# посмотрим на дубли
data_stage_2.duplicated().sum()

3984

In [37]:
data_stage_3 = data_stage_2.drop_duplicates()
data_stage_3.head()

,customer_id,order_id,payment_value_x,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_zip_code_prefix,customer_city,customer_state
0,00012a2ce6f8dcda20d059ce98491703,5f79b5b0931d63f1a42989eb65b9da6e,114.74,delivered,2017-11-14 16:08:26,2017-11-14 16:35:32,2017-11-17 15:32:08,2017-11-28 15:41:30,2017-12-04 00:00:00,1,...,889.0,1.0,4267.0,46.0,39.0,22.0,toys,6273,osasco,SP
1,000161a058600d5901f007fab4c27140,a44895d095d7e0702b6a162fa2dbeced,67.41,delivered,2017-07-16 09:40:32,2017-07-16 09:55:12,2017-07-19 19:09:37,2017-07-25 18:57:33,2017-08-04 00:00:00,1,...,2563.0,2.0,150.0,16.0,2.0,11.0,health_beauty,35550,itapecerica,MG
2,0001fd6190edaaf884bcaf3d49edf079,316a104623542e4d75189bb372bc5f8d,195.42,delivered,2017-02-28 11:06:43,2017-02-28 11:15:20,2017-03-01 15:24:20,2017-03-06 08:57:49,2017-03-22 00:00:00,1,...,1042.0,2.0,750.0,21.0,23.0,35.0,baby,29830,nova venecia,ES
3,0002414f95344307404f0ace7a26f1d5,5825ce2e88d5346438686b0bba99e5ee,179.35,delivered,2017-08-16 13:09:20,2017-08-17 03:10:27,2017-08-19 11:34:29,2017-09-13 20:06:02,2017-09-14 00:00:00,0,...,840.0,1.0,1600.0,40.0,24.0,30.0,cool_stuff,39664,mendonca,MG
4,000379cdec625522490c315e70c7a9fb,0ab7fb08086d4af9141453c91878ed7a,107.01,delivered,2018-04-02 13:42:17,2018-04-04 03:10:19,2018-04-04 18:11:09,2018-04-13 20:21:08,2018-04-18 00:00:00,0,...,364.0,2.0,800.0,45.0,10.0,34.0,bed_bath_table,4841,sao paulo,SP


In [47]:
data_stage_4 = data_stage_3.dropna()


In [49]:
data_stage_4.isna().sum()


customer_id                      0
order_id                         0
payment_value_x                  0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
payment_installments             0
review_score                     0
review_creation_date             0
review_answer_timestamp          0
order_item_id                    0
shipping_limit_date              0
price                            0
freight_value                    0
seller_zip_code_prefix           0
seller_city                      0
seller_state                     0
product_category_name            0
product_name_lenght              0
product_description_lenght       0
product_photos_qty               0
product_weight_g                 0
product_length_cm                0
product_height_cm                0
product_width_cm                 0
product_category_nam

In [50]:
data_stage_4.duplicated().sum()

0

In [52]:
data_stage_4

,customer_id,order_id,payment_value_x,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_zip_code_prefix,customer_city,customer_state
0,00012a2ce6f8dcda20d059ce98491703,5f79b5b0931d63f1a42989eb65b9da6e,114.74,delivered,2017-11-14 16:08:26,2017-11-14 16:35:32,2017-11-17 15:32:08,2017-11-28 15:41:30,2017-12-04 00:00:00,1,...,889.0,1.0,4267.0,46.0,39.0,22.0,toys,6273,osasco,SP
1,000161a058600d5901f007fab4c27140,a44895d095d7e0702b6a162fa2dbeced,67.41,delivered,2017-07-16 09:40:32,2017-07-16 09:55:12,2017-07-19 19:09:37,2017-07-25 18:57:33,2017-08-04 00:00:00,1,...,2563.0,2.0,150.0,16.0,2.0,11.0,health_beauty,35550,itapecerica,MG
2,0001fd6190edaaf884bcaf3d49edf079,316a104623542e4d75189bb372bc5f8d,195.42,delivered,2017-02-28 11:06:43,2017-02-28 11:15:20,2017-03-01 15:24:20,2017-03-06 08:57:49,2017-03-22 00:00:00,1,...,1042.0,2.0,750.0,21.0,23.0,35.0,baby,29830,nova venecia,ES
3,0002414f95344307404f0ace7a26f1d5,5825ce2e88d5346438686b0bba99e5ee,179.35,delivered,2017-08-16 13:09:20,2017-08-17 03:10:27,2017-08-19 11:34:29,2017-09-13 20:06:02,2017-09-14 00:00:00,0,...,840.0,1.0,1600.0,40.0,24.0,30.0,cool_stuff,39664,mendonca,MG
4,000379cdec625522490c315e70c7a9fb,0ab7fb08086d4af9141453c91878ed7a,107.01,delivered,2018-04-02 13:42:17,2018-04-04 03:10:19,2018-04-04 18:11:09,2018-04-13 20:21:08,2018-04-18 00:00:00,0,...,364.0,2.0,800.0,45.0,10.0,34.0,bed_bath_table,4841,sao paulo,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115699,fffecc9f79fd8c764f843e9951b11341,814d6a3a7c0b32b2ad929ac6328124e9,81.36,delivered,2018-03-29 16:59:26,2018-03-29 17:10:27,2018-03-31 14:29:38,2018-04-10 17:20:49,2018-04-27 00:00:00,0,...,641.0,3.0,2050.0,50.0,10.0,40.0,bed_bath_table,95630,parobe,RS
115702,fffeda5b6d849fbd39689bb92087f431,8c855550908247a7eff50281b92167a8,63.13,delivered,2018-05-22 13:36:02,2018-05-22 13:54:37,2018-05-25 13:25:00,2018-06-08 18:03:31,2018-06-29 00:00:00,1,...,458.0,4.0,350.0,20.0,30.0,20.0,telephony,22461,rio de janeiro,RJ
115703,ffff42319e9b2d713724ae527742af25,83b5fc912b2862c5046555ded1483ae9,214.13,delivered,2018-06-13 16:57:05,2018-06-13 17:20:23,2018-06-15 18:52:00,2018-06-18 18:33:05,2018-06-25 00:00:00,1,...,268.0,1.0,2700.0,41.0,8.0,36.0,bed_bath_table,6754,taboao da serra,SP
115704,ffffa3172527f765de70084a7e53aae8,d0e7be325a1c986babc4e1cdb91edc03,91.00,delivered,2017-09-02 11:53:32,2017-09-02 12:05:40,2017-09-08 20:04:11,2017-09-14 19:47:40,2017-09-26 00:00:00,1,...,161.0,3.0,200.0,20.0,20.0,20.0,furniture_decor,37130,alfenas,MG


In [55]:
data_stage_4[data_stage_4["order_id"].duplicated()]

,customer_id,order_id,payment_value_x,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_zip_code_prefix,customer_city,customer_state
13,00066ccbe787a588c52bd5ff404590e3,728416b0db65935dbf78a0cc03e8d6f8,1080.00,delivered,2018-02-06 16:10:09,2018-02-08 07:49:51,2018-02-14 19:58:58,2018-02-21 22:22:17,2018-03-12 00:00:00,0,...,366.0,2.0,1650.0,30.0,22.0,30.0,garden_tools,93525,novo hamburgo,RS
14,00066ccbe787a588c52bd5ff404590e3,728416b0db65935dbf78a0cc03e8d6f8,1080.00,delivered,2018-02-06 16:10:09,2018-02-08 07:49:51,2018-02-14 19:58:58,2018-02-21 22:22:17,2018-03-12 00:00:00,0,...,366.0,2.0,1650.0,30.0,22.0,30.0,garden_tools,93525,novo hamburgo,RS
15,00066ccbe787a588c52bd5ff404590e3,728416b0db65935dbf78a0cc03e8d6f8,1080.00,delivered,2018-02-06 16:10:09,2018-02-08 07:49:51,2018-02-14 19:58:58,2018-02-21 22:22:17,2018-03-12 00:00:00,0,...,366.0,2.0,1650.0,30.0,22.0,30.0,garden_tools,93525,novo hamburgo,RS
19,000bf8121c3412d3057d32371c5d3395,bc3e295306ee4d3eba91aca49b0bb539,91.12,delivered,2017-10-11 07:44:31,2017-10-11 07:56:17,2017-10-13 15:16:26,2017-10-16 19:35:33,2017-10-24 00:00:00,0,...,431.0,4.0,300.0,20.0,5.0,48.0,furniture_decor,12335,jacarei,SP
38,001450ebb4a77efb3d68be5f7887cb1e,41a75aec932b24ba50e98ba8b50b625d,122.66,delivered,2018-02-26 08:35:13,2018-02-27 08:30:28,2018-03-05 14:19:44,2018-03-13 18:22:54,2018-03-19 00:00:00,1,...,451.0,2.0,185.0,24.0,7.0,16.0,telephony,18150,ibiuna,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115673,fff168ca1f8a1d2e8e2108b231a68a8c,09ec142bfa34576d3914bdf8c19927c2,212.28,delivered,2018-07-20 16:25:59,2018-07-20 16:44:13,2018-07-27 10:28:00,2018-07-30 18:26:39,2018-08-01 00:00:00,0,...,399.0,4.0,950.0,40.0,15.0,31.0,furniture_decor,3167,sao paulo,SP
115674,fff168ca1f8a1d2e8e2108b231a68a8c,09ec142bfa34576d3914bdf8c19927c2,212.28,delivered,2018-07-20 16:25:59,2018-07-20 16:44:13,2018-07-27 10:28:00,2018-07-30 18:26:39,2018-08-01 00:00:00,0,...,400.0,4.0,750.0,30.0,15.0,30.0,furniture_decor,3167,sao paulo,SP
115694,fff93c1da78dafaaa304ff032abc6205,2bedc0d7231e504d0476c606108e7e73,727.44,delivered,2018-06-13 01:57:22,2018-06-13 02:59:41,2018-06-18 14:01:00,2018-06-21 11:56:46,2018-07-11 00:00:00,1,...,1043.0,1.0,1050.0,30.0,4.0,20.0,books_general_interest,14802,araraquara,SP
115695,fff93c1da78dafaaa304ff032abc6205,2bedc0d7231e504d0476c606108e7e73,727.44,delivered,2018-06-13 01:57:22,2018-06-13 02:59:41,2018-06-18 14:01:00,2018-06-21 11:56:46,2018-07-11 00:00:00,1,...,1055.0,1.0,600.0,30.0,4.0,20.0,books_general_interest,14802,araraquara,SP


> дублируется из-за order_item_id 

In [57]:
data_stage_5 = data_stage_4.drop(columns=["order_item_id"])

In [59]:
data_stage_6 = data_stage_5.drop_duplicates()

In [60]:
data_stage_6[data_stage_6["order_id"].duplicated()]

,customer_id,order_id,payment_value_x,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_zip_code_prefix,customer_city,customer_state
38,001450ebb4a77efb3d68be5f7887cb1e,41a75aec932b24ba50e98ba8b50b625d,122.66,delivered,2018-02-26 08:35:13,2018-02-27 08:30:28,2018-03-05 14:19:44,2018-03-13 18:22:54,2018-03-19 00:00:00,1,...,451.0,2.0,185.0,24.0,7.0,16.0,telephony,18150,ibiuna,SP
75,002f90a6eb386bc43bc9ba200db31a89,a6c173f716aac4c915299b74fb4e0b10,307.02,delivered,2017-06-15 23:59:20,2017-06-16 00:55:21,2017-06-16 13:49:35,2017-06-23 12:57:41,2017-07-12 00:00:00,1,...,864.0,6.0,350.0,18.0,18.0,21.0,consoles_games,4929,sao paulo,SP
84,0031abfb953b66e998f67b09e7b11375,47dc3a1de5a0aec4323cdade0588655c,1343.70,delivered,2018-01-23 13:57:55,2018-01-24 15:07:56,2018-01-25 23:21:17,2018-02-01 20:53:04,2018-02-20 00:00:00,1,...,220.0,1.0,750.0,23.0,2.0,20.0,watches_gifts,84040,ponta grossa,PR
91,0033823ee55671ac5317d423291333c2,3ea8ef64da06a959e384edb7cb824c02,158.96,delivered,2017-05-29 19:02:27,2017-05-29 19:15:14,2017-06-01 10:56:42,2017-06-12 11:33:43,2017-06-23 00:00:00,0,...,1087.0,1.0,600.0,28.0,11.0,26.0,health_beauty,95041,caxias do sul,RS
127,00474d2582fd72663036795b7ab8cfc1,8b388d845a3e8700444b971fa2bef902,217.68,delivered,2016-10-04 22:33:44,2016-10-06 03:10:59,2016-10-28 13:28:35,2016-11-08 14:03:58,2016-12-06 00:00:00,0,...,60.0,1.0,200.0,30.0,20.0,30.0,baby,87600,nova esperanca,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115585,ffbcf1eb664ce45bed8883fbcbdc2b06,f1b0d64e31c26320c0447cd14d66f4bf,285.14,delivered,2017-03-03 01:23:13,2017-03-03 01:30:14,2017-03-03 14:05:12,2017-03-20 07:06:11,2017-04-05 00:00:00,1,...,589.0,4.0,200.0,16.0,7.0,11.0,toys,65725,pedreiras,MA
115611,ffc436ef9df6ede268ee1548a0dfbbba,11156cbbd16c21d0f876588915098a28,592.20,delivered,2018-04-25 09:00:27,2018-04-25 09:15:13,2018-04-26 13:05:00,2018-05-10 00:25:27,2018-05-16 00:00:00,1,...,978.0,3.0,1450.0,25.0,15.0,25.0,computers_accessories,24724,sao goncalo,RJ
115626,ffd094af4643c7ac712acd36a56feccc,7ca8f2556a9744a905d8f831fe60e39e,316.06,delivered,2018-05-13 21:41:04,2018-05-15 04:35:11,2018-05-15 13:25:00,2018-05-22 12:16:32,2018-06-06 00:00:00,0,...,581.0,1.0,150.0,16.0,7.0,15.0,health_beauty,91751,porto alegre,RS
115673,fff168ca1f8a1d2e8e2108b231a68a8c,09ec142bfa34576d3914bdf8c19927c2,212.28,delivered,2018-07-20 16:25:59,2018-07-20 16:44:13,2018-07-27 10:28:00,2018-07-30 18:26:39,2018-08-01 00:00:00,0,...,399.0,4.0,950.0,40.0,15.0,31.0,furniture_decor,3167,sao paulo,SP


In [64]:
data_stage_6 = data_stage_5.rename(columns={"payment_value_x": "payment_sum"})

In [65]:
data_stage_6.head(60)

,customer_id,order_id,payment_sum,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_zip_code_prefix,customer_city,customer_state
0,00012a2ce6f8dcda20d059ce98491703,5f79b5b0931d63f1a42989eb65b9da6e,114.74,delivered,2017-11-14 16:08:26,2017-11-14 16:35:32,2017-11-17 15:32:08,2017-11-28 15:41:30,2017-12-04 00:00:00,1,...,889.0,1.0,4267.0,46.0,39.0,22.0,toys,6273,osasco,SP
1,000161a058600d5901f007fab4c27140,a44895d095d7e0702b6a162fa2dbeced,67.41,delivered,2017-07-16 09:40:32,2017-07-16 09:55:12,2017-07-19 19:09:37,2017-07-25 18:57:33,2017-08-04 00:00:00,1,...,2563.0,2.0,150.0,16.0,2.0,11.0,health_beauty,35550,itapecerica,MG
2,0001fd6190edaaf884bcaf3d49edf079,316a104623542e4d75189bb372bc5f8d,195.42,delivered,2017-02-28 11:06:43,2017-02-28 11:15:20,2017-03-01 15:24:20,2017-03-06 08:57:49,2017-03-22 00:00:00,1,...,1042.0,2.0,750.0,21.0,23.0,35.0,baby,29830,nova venecia,ES
3,0002414f95344307404f0ace7a26f1d5,5825ce2e88d5346438686b0bba99e5ee,179.35,delivered,2017-08-16 13:09:20,2017-08-17 03:10:27,2017-08-19 11:34:29,2017-09-13 20:06:02,2017-09-14 00:00:00,0,...,840.0,1.0,1600.0,40.0,24.0,30.0,cool_stuff,39664,mendonca,MG
4,000379cdec625522490c315e70c7a9fb,0ab7fb08086d4af9141453c91878ed7a,107.01,delivered,2018-04-02 13:42:17,2018-04-04 03:10:19,2018-04-04 18:11:09,2018-04-13 20:21:08,2018-04-18 00:00:00,0,...,364.0,2.0,800.0,45.0,10.0,34.0,bed_bath_table,4841,sao paulo,SP
5,0004164d20a9e969af783496f3408652,cd3558a10d854487b4f907e9b326a4fc,71.80,delivered,2017-04-12 08:35:12,2017-04-12 08:50:12,2017-04-12 17:05:42,2017-04-20 16:12:26,2017-05-04 00:00:00,0,...,475.0,4.0,800.0,50.0,15.0,25.0,sports_leisure,13272,valinhos,SP
6,000419c5494106c306a97b5635748086,07f6c3baf9ac86865b60f640c4f923c6,49.40,delivered,2018-03-02 17:47:40,2018-03-03 14:10:38,2018-03-07 21:07:51,2018-04-17 17:17:34,2018-03-22 00:00:00,1,...,1359.0,1.0,250.0,16.0,2.0,11.0,fashion_bags_accessories,24220,niteroi,RJ
7,00046a560d407e99b969756e0b10f282,8c3d752c5c02227878fae49aeaddbfd7,166.59,delivered,2017-12-18 11:08:30,2017-12-18 12:45:31,2017-12-18 20:55:54,2017-12-26 20:58:33,2018-01-12 00:00:00,1,...,707.0,5.0,7050.0,20.0,62.0,35.0,toys,20540,rio de janeiro,RJ
8,00050bf6e01e69d5c0fd612f1bcfb69c,fa906f338cee30a984d0945b3832e431,85.23,delivered,2017-09-17 16:04:44,2017-09-17 16:15:13,2017-09-18 21:02:46,2017-10-02 21:14:31,2017-10-13 00:00:00,1,...,874.0,3.0,200.0,20.0,10.0,20.0,fashion_bags_accessories,98700,ijui,RS
9,000598caf2ef4117407665ac33275130,9b961b894e797f63622137ff7eb1c1af,1255.71,delivered,2018-08-11 12:14:35,2018-08-11 12:25:08,2018-08-13 14:41:00,2018-08-20 18:18:41,2018-09-06 00:00:00,1,...,1308.0,4.0,20550.0,69.0,28.0,56.0,pet_shop,35540,oliveira,MG


> Дублируется за счет характеристик товара, в одном заказе могло быть несколько товаров по разной цене

----
### Наченем конфигурировать датасет для непосредственной кластеризации клиентов

In [95]:
# Расчитаем суммарное количество заказов на пользователя
clust_df = data_stage_6.groupby(
    "customer_id")["order_id"].count().reset_index()
clust_df = clust_df.rename(columns={"order_id": "num_of_orders"})
clust_df

,customer_id,num_of_orders
0,00012a2ce6f8dcda20d059ce98491703,1
1,000161a058600d5901f007fab4c27140,1
2,0001fd6190edaaf884bcaf3d49edf079,1
3,0002414f95344307404f0ace7a26f1d5,1
4,000379cdec625522490c315e70c7a9fb,1
...,...,...
94467,fffcb937e9dd47a13f05ecb8290f4d3e,1
94468,fffecc9f79fd8c764f843e9951b11341,1
94469,fffeda5b6d849fbd39689bb92087f431,1
94470,ffff42319e9b2d713724ae527742af25,1


In [96]:
# Итоговая сумма по заказам
payment_sum = data_stage_6.groupby(
    "customer_id")["payment_sum"].sum().reset_index()

clust_df = pd.merge(left=clust_df, right=payment_sum, on="customer_id", how="left")
clust_df

,customer_id,num_of_orders,payment_sum
0,00012a2ce6f8dcda20d059ce98491703,1,114.74
1,000161a058600d5901f007fab4c27140,1,67.41
2,0001fd6190edaaf884bcaf3d49edf079,1,195.42
3,0002414f95344307404f0ace7a26f1d5,1,179.35
4,000379cdec625522490c315e70c7a9fb,1,107.01
...,...,...,...
94467,fffcb937e9dd47a13f05ecb8290f4d3e,1,91.91
94468,fffecc9f79fd8c764f843e9951b11341,1,81.36
94469,fffeda5b6d849fbd39689bb92087f431,1,63.13
94470,ffff42319e9b2d713724ae527742af25,1,214.13


In [97]:
data_stage_6["order_status"].value_counts()

order_status
delivered    109307
canceled          7
Name: count, dtype: int64

> order_status нет смысла смотреть

In [103]:
first_order = data_stage_6.groupby(
    "customer_id")["order_purchase_timestamp"].min().reset_index().rename(columns={"order_purchase_timestamp": "first_order"})
last_order = data_stage_6.groupby(
    "customer_id")["order_purchase_timestamp"].max().reset_index().rename(columns={"order_purchase_timestamp": "last_order"})

orders_dates = pd.merge(left=first_order, right=last_order, on="customer_id")
orders_dates["first_order"] = pd.to_datetime(orders_dates["first_order"])
orders_dates["last_order"] = pd.to_datetime(orders_dates["last_order"])

orders_dates["life_span"] = orders_dates["last_order"] - orders_dates["first_order"]
orders_dates

,customer_id,first_order,last_order,life_span
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,2017-11-14 16:08:26,0 days
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,2017-07-16 09:40:32,0 days
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43,2017-02-28 11:06:43,0 days
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,2017-08-16 13:09:20,0 days
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,2018-04-02 13:42:17,0 days
...,...,...,...,...
94467,fffcb937e9dd47a13f05ecb8290f4d3e,2018-03-17 00:55:27,2018-03-17 00:55:27,0 days
94468,fffecc9f79fd8c764f843e9951b11341,2018-03-29 16:59:26,2018-03-29 16:59:26,0 days
94469,fffeda5b6d849fbd39689bb92087f431,2018-05-22 13:36:02,2018-05-22 13:36:02,0 days
94470,ffff42319e9b2d713724ae527742af25,2018-06-13 16:57:05,2018-06-13 16:57:05,0 days


In [116]:
clust_df[clust_df["num_of_orders"] > 10]

,customer_id,num_of_orders,payment_sum
4978,0d93f21f3e8543a9d0d8ece01561f5b2,12,63607.68
6199,10de381f8a8d23fff822753305f71cae,15,275771.25
21993,3b54b5978e9ace64a63f90d176ffb158,12,43562.88
26872,48d6a6909a9bd0b2128ea79630e52d13,12,104829.12
46177,7d321bd4e8ba1caf74c4c1aabd9ae524,13,82123.86
51640,8c20d9bfbc96c5d39025d77a3ba83d7f,11,240614.55
53838,91f92cfee46b79581b05aa974dd57ce5,12,213442.56
58609,9eb3d566e87289dcb0acf28e1407c839,12,35138.88
61802,a7693fba2ff9583c78751f2b66ecab9d,14,103640.88
64258,adb32467ecc74b53576d9d13a5a55891,15,176175.00


In [111]:
orders_dates[orders_dates["customer_id"] == "fc3d1daec319d62d49bfb5e1f83123e9"]

,customer_id,first_order,last_order,life_span
93100,fc3d1daec319d62d49bfb5e1f83123e9,2017-07-16 18:19:25,2017-07-16 18:19:25,0 days


In [117]:
data_stage_6[data_stage_6["customer_id"] == "0d93f21f3e8543a9d0d8ece01561f5b2"]

,customer_id,order_id,payment_sum,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_zip_code_prefix,customer_city,customer_state
6114,0d93f21f3e8543a9d0d8ece01561f5b2,2c2a19b5703863c908512d135aa6accc,5300.64,delivered,2017-10-09 20:45:45,2017-10-11 03:26:49,2017-10-11 21:14:36,2017-10-18 16:19:17,2017-10-27 00:00:00,0,...,465.0,1.0,550.0,16.0,22.0,12.0,housewares,36025,juiz de fora,MG
6115,0d93f21f3e8543a9d0d8ece01561f5b2,2c2a19b5703863c908512d135aa6accc,5300.64,delivered,2017-10-09 20:45:45,2017-10-11 03:26:49,2017-10-11 21:14:36,2017-10-18 16:19:17,2017-10-27 00:00:00,0,...,465.0,1.0,550.0,16.0,22.0,12.0,housewares,36025,juiz de fora,MG
6116,0d93f21f3e8543a9d0d8ece01561f5b2,2c2a19b5703863c908512d135aa6accc,5300.64,delivered,2017-10-09 20:45:45,2017-10-11 03:26:49,2017-10-11 21:14:36,2017-10-18 16:19:17,2017-10-27 00:00:00,0,...,465.0,1.0,550.0,16.0,22.0,12.0,housewares,36025,juiz de fora,MG
6117,0d93f21f3e8543a9d0d8ece01561f5b2,2c2a19b5703863c908512d135aa6accc,5300.64,delivered,2017-10-09 20:45:45,2017-10-11 03:26:49,2017-10-11 21:14:36,2017-10-18 16:19:17,2017-10-27 00:00:00,0,...,465.0,1.0,550.0,16.0,22.0,12.0,housewares,36025,juiz de fora,MG
6118,0d93f21f3e8543a9d0d8ece01561f5b2,2c2a19b5703863c908512d135aa6accc,5300.64,delivered,2017-10-09 20:45:45,2017-10-11 03:26:49,2017-10-11 21:14:36,2017-10-18 16:19:17,2017-10-27 00:00:00,0,...,465.0,1.0,550.0,16.0,22.0,12.0,housewares,36025,juiz de fora,MG
6119,0d93f21f3e8543a9d0d8ece01561f5b2,2c2a19b5703863c908512d135aa6accc,5300.64,delivered,2017-10-09 20:45:45,2017-10-11 03:26:49,2017-10-11 21:14:36,2017-10-18 16:19:17,2017-10-27 00:00:00,0,...,465.0,1.0,550.0,16.0,22.0,12.0,housewares,36025,juiz de fora,MG
6120,0d93f21f3e8543a9d0d8ece01561f5b2,2c2a19b5703863c908512d135aa6accc,5300.64,delivered,2017-10-09 20:45:45,2017-10-11 03:26:49,2017-10-11 21:14:36,2017-10-18 16:19:17,2017-10-27 00:00:00,0,...,465.0,1.0,550.0,16.0,22.0,12.0,housewares,36025,juiz de fora,MG
6121,0d93f21f3e8543a9d0d8ece01561f5b2,2c2a19b5703863c908512d135aa6accc,5300.64,delivered,2017-10-09 20:45:45,2017-10-11 03:26:49,2017-10-11 21:14:36,2017-10-18 16:19:17,2017-10-27 00:00:00,0,...,465.0,1.0,550.0,16.0,22.0,12.0,housewares,36025,juiz de fora,MG
6122,0d93f21f3e8543a9d0d8ece01561f5b2,2c2a19b5703863c908512d135aa6accc,5300.64,delivered,2017-10-09 20:45:45,2017-10-11 03:26:49,2017-10-11 21:14:36,2017-10-18 16:19:17,2017-10-27 00:00:00,0,...,465.0,1.0,550.0,16.0,22.0,12.0,housewares,36025,juiz de fora,MG
6123,0d93f21f3e8543a9d0d8ece01561f5b2,2c2a19b5703863c908512d135aa6accc,5300.64,delivered,2017-10-09 20:45:45,2017-10-11 03:26:49,2017-10-11 21:14:36,2017-10-18 16:19:17,2017-10-27 00:00:00,0,...,465.0,1.0,550.0,16.0,22.0,12.0,housewares,36025,juiz de fora,MG


In [119]:
test = pd.read_csv("./clean_data/customers.csv")
test

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS
